In [1]:
#### 90 samples
ind_dict = {
            "individual_1": ['brain_IP_1', 'heart_IP_1', 'liver_IP_1'],  
            "individual_2": ['brain_IP_2', 'heart_IP_2', 'placenta_IP_2', 'kidney_IP_2', 'liver_IP_2'],  
            "individual_3": ['brain_IP_3', 'heart_IP_3', 'liver_IP_3', 'kidney_IP_3'], 
            "individual_4": ['stomach_IP_4', 'muscle_IP_4', 'lung_IP_4', 'kidney_IP_4', 'placenta_IP_4'], 
            "individual_5": ['stomach_IP_5', 'muscle_IP_5', 'lung_IP_5'], 
            "individual_6": ['placenta_IP_6'],
            "individual_7": ['Adipose-1-1', 'Adrenal_gland-1-1', 'Aorta-1-1', 'Heart-1-1', 'Skin-1-1', 'Spleen-1-1'], 
            "individual_8": ['Lung-2-1', 'Lung-2-4', 'Spleen-2-1', 'Tongue-2-1', 'Urinary_bladder-2-1'], 
            "individual_9": ['Appendix-3-2', 'Colon-3-2', 'Esophagus-3-2', 'Muscle-3-2', 'Spleen-3-2'],
            "individual_10": ['Aorta-4-2', 'Esophagus-4-2', 'Heart-4-2', 'Jejunum-4-2', 'Liver-4-2','Lung-4-2','Lung-4-4','Prostate-4-2','Rectum-4-2','Skin-4-2','Stomach-4-2','Testis-4-2','Thyroid_gland-4-2','Urinary_bladder-4-2'],
            "individual_11": ['Appendix-5-3', 'Brainstem-5-3', 'Cerebellum-5-3', 'Cerebrum-5-3', 'Duodenum-5-3', 'Hypothalamus-5-3','Jejunum-5-3','Muscle-5-3','Rectum-5-3','Stomach-5-3','Thyroid_gland-5-3','Trachea-5-3','Urinary_bladder-5-3'], 
            "individual_12": ['Cerebrum-6-3'],
            "individual_13": ['Cerebellum-7-4'],
            "individual_14": ['FrontalCortex_1','Cerebellum_1','Heart_1','Liver_1','Lung_1','Kidney_1','Spleen_1'],
            "individual_15": ['FrontalCortex_2','Cerebellum_2','Heart_2','Liver_2','Kidney_2','Spleen_2','Muscle_1'],
            "individual_16": ['FrontalCortex_3','Cerebellum_3','Heart_3','Liver_3','Lung_2','Muscle_2'],
            "individual_17": ['Muscle_3'],
            "individual_18": ['Lung_3','Kidney_3','Spleen_3']
           }

# tissue_dict = {
#             "liver": ["liver_IP_1.bed",'liver_IP_2.bed','liver_IP_3.bed','Liver-4-2.bed','Liver_1.bed','Liver_2.bed','Liver_3.bed'],
#             "muscle": ["muscle_IP_4.bed",'muscle_IP_5.bed','Muscle-3-2.bed','Muscle-5-3.bed','Muscle_1.bed','Muscle_2.bed','Muscle_3.bed'],
#             "lung": ['lung_IP_4.bed','lung_IP_5.bed','Lung-2-1.bed','Lung-2-4.bed','Lung-4-2.bed','Lung-4-4.bed','Lung_1.bed','Lung_2.bed','Lung_3.bed'],
#             "heart": ["heart_IP_1.bed",'heart_IP_2.bed','heart_IP_3.bed','Heart-1-1.bed','Heart-4-2.bed','Heart_1.bed','Heart_2.bed','Heart_3.bed'],
#             "brain": ["brain_IP_1.bed",'brain_IP_2.bed','brain_IP_3.bed','Brainstem-5-3.bed','Cerebellum-5-3.bed','Cerebellum-7-4.bed','Cerebrum-5-3.bed','Cerebrum-6-3.bed','Hypothalamus-5-3.bed','Cerebellum_1.bed','Cerebellum_2.bed','Cerebellum_3.bed','FrontalCortex_1.bed','FrontalCortex_2.bed','FrontalCortex_3.bed']
#             }

In [40]:
### select heterozygous SNPs shared in two samples

def extract_het(txt):
    df = pd.read_table(txt)
    df['term'] = df['contig'] + ":" + df['position'].astype(str)
    return list(set(df['term'].tolist()))

def read_txt(txt):
    prefix = os.path.basename(txt).split(".bed")[0]
    df = pd.read_table(txt, header=None)
    # df['term'] = df['contig'] + ":" + df['position'].astype(str)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    # df['mark_%s'%prefix] = np.where(df['refRPKM_ratio']>df['altRPKM_ratio'], 'ref', 'alt')
    df['mark_%s'%prefix] = df.iloc[:,3].str.split(";").str[2]
    return df[['term','mark_%s'%prefix]].drop_duplicates()

def get_index_sample(sample1_snp_list,sample2_snp_list):
    sample1_intersection_sample2 = list(set(sample1_snp_list).intersection(set(sample2_snp_list)))
    sample1_union_sample2 = list(set(sample1_snp_list).union(set(sample2_snp_list)))
    if len(sample1_union_sample2) == 0:
        jaccard_index = 0
    else:
        jaccard_index = len(sample1_intersection_sample2)/len(sample1_union_sample2)
    return jaccard_index

def calc_jarcard_index(sample_1, sample_2):
    hets_1, hets_2 = extract_het(sample_1), extract_het(sample_2)
    common_hets = list(set(hets_1).intersection(set(hets_2)))
#     print("Total common hets: %d" % len(common_hets))
    ### ASm6As exist in common samples
    asm6a_1 = os.path.join(asm6a_dir, sample_1.replace(".txt",".bed"))
    asm6a_2 = os.path.join(asm6a_dir, sample_2.replace(".txt",".bed"))
    df_1, df_2 = read_txt(asm6a_1), read_txt(asm6a_2)
    df_1 = df_1[df_1['term'].isin(common_hets)]
    df_2 = df_2[df_2['term'].isin(common_hets)]
    asm6a_1_list = df_1['term'].tolist()
    asm6a_2_list = df_2['term'].tolist()
    jar_index = get_index_sample(asm6a_1_list, asm6a_2_list)
    return jar_index
#     print(sample_1, sample_2, jar_index)
#     df = df_1.merge(df_2, on='term', how="outer")
#     df = df[df['term'].isin(common_hets)]
#     print("ASm6As deposted on the 'shared' het Sites: %d" % len(df))
#     ###
#     df_com = df.dropna(how="any")
#     print("#######\nCommon ASm6As: %d" % len(df_com))
#     df_com_s = df_com[df_com.iloc[:,1] == df_com.iloc[:,2]]
#     df_com_o = df_com[df_com.iloc[:,1] != df_com.iloc[:,2]]
#     print("same trend: %d\noppo trend: %d\n########" % (len(df_com_s), len(df_com_o)))
#     df_non = df[(df.iloc[:,1].isna()) | (df.iloc[:,2].isna())]
#     print("Not ASm6A in either sample: %d" % len(df_non))


# asm6a_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/sig/contained_m6A/rename/"
asm6a_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/sig/contained_m6A/highFC/rename/"
os.chdir("/Charles/project/ASm6A/Hypothesis_test/ASm6A/rename/")
### These sites contained in files only satisfy the minimun read counts
result_dir = "/Charles/project/ASm6A/ASm6A/by_tissue/jarcard_index/"
result_file = os.path.join(result_dir, "fetal_index.txt") # fetal_index

tissue_list = ["brain", 'heart', 'liver', 'lung','kidney','muscle']
bed_list = [x for x in glob.glob("*IP*.txt") if x.split("_")[0] in tissue_list]
# bed_list = [x for x in glob.glob("*.txt") if "IP" not in x]
sample_list = [x.split(".")[0] for x in bed_list]
with open(result_file, 'w') as fw:
    fw.write("\t".join(["Sample"] + sample_list) + "\n")
    for bed in bed_list:
        index_list = []
        for match_bed in bed_list:
            index= calc_jarcard_index(bed, match_bed)
            index_list.append(index)
        # print("\t".join([bed] + [str(x) for x in index_list]))
        fw.write("\t".join([bed.split(".")[0]] + [str(x) for x in index_list]) + "\n")